In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import time as ttime
import pandas as pd
import os
import matplotlib.colors as mcolors
from selenium.webdriver.common.by import By
from moviepy.editor import *
import random

In [2]:
def clica_agree(driver):
    #Localiza botao Agree e clica
    elem = driver.find_element('xpath','/html/body/div[1]/div/div/div/div[2]/div/button[3]')
    actions = ActionChains(driver)
    actions.click(elem).perform()

def clicar(driver,xpath):
    #Localiza botao Agree e clica
    elem = driver.find_element('xpath',xpath)
    actions = ActionChains(driver)
    actions.click(elem).perform()

def ler_tabela(driver,xpath_c, linha_c,coluna_c,xpath_b,linha_b,coluna_b):
    df = pd.DataFrame()

    #Cabeçalho
    tabela = driver.find_element("xpath",xpath_c)
    linhas = tabela.find_elements(By.TAG_NAME,linha_c) 
    index = []
    for linha in linhas:
        colunas = linha.find_elements(By.TAG_NAME,coluna_c)
        col = []
        for coluna in colunas:
            index.append(coluna.text)

    #Body
    tabela = driver.find_element("xpath",xpath_b)
    linhas = tabela.find_elements(By.TAG_NAME,linha_b)

    for linha in linhas:
        colunas = linha.find_elements(By.TAG_NAME,coluna_b)
        col = []
        for coluna in colunas:
            col.append(coluna.text)

        df[col[0]] = col[1:]

    df.index = index

    df = df.T

    return df

def transf_informacao_maior_melhor(df,coluna_nova,coluna_variavel):
    df[coluna_nova] = ''
    parcela = (df[coluna_variavel].max()-df[coluna_variavel].min())/5
    df[coluna_variavel].min()+parcela
    q1 = df[coluna_variavel].min()+1*parcela
    q2 = df[coluna_variavel].min()+2*parcela
    q3 = df[coluna_variavel].min()+3*parcela
    q4 = df[coluna_variavel].min()+4*parcela

    df.loc[df[coluna_variavel]>q4,coluna_nova] = 2
    df.loc[(df[coluna_variavel]>q3)&(df[coluna_variavel]<q4),coluna_nova] = 1
    df.loc[(df[coluna_variavel]>q2)&(df[coluna_variavel]<q3),coluna_nova] = 0
    df.loc[(df[coluna_variavel]>q1)&(df[coluna_variavel]<q2),coluna_nova] = -1
    df.loc[(df[coluna_variavel]<q1),coluna_nova] = -2
    return df

def transf_informacao_maior_pior(df,coluna_nova,coluna_variavel):
    df[coluna_nova] = ''
    parcela = (df[coluna_variavel].max()-df[coluna_variavel].min())/5
    df[coluna_variavel].min()+parcela
    q1 = df[coluna_variavel].min()+1*parcela
    q2 = df[coluna_variavel].min()+2*parcela
    q3 = df[coluna_variavel].min()+3*parcela
    q4 = df[coluna_variavel].min()+4*parcela

    df.loc[df[coluna_variavel]>q4,coluna_nova] = -2
    df.loc[(df[coluna_variavel]>q3)&(df[coluna_variavel]<q4),coluna_nova] = -1
    df.loc[(df[coluna_variavel]>q2)&(df[coluna_variavel]<q3),coluna_nova] = 0
    df.loc[(df[coluna_variavel]>q1)&(df[coluna_variavel]<q2),coluna_nova] = 1
    df.loc[(df[coluna_variavel]<q1),coluna_nova] = 2
    return df

def jogos_em_casa(valor):
    if '-' in valor:
        return int(valor.split(' - ')[0]) - int(valor.split(' - ')[1])
    return valor

def jogos_fora(valor):
    if '-' in valor:
        return int(valor.split(' - ')[1]) - int(valor.split(' - ')[0])
    return valor

def jogos_em_casa_gols_marcados(valor):
    if '-' in valor:
        return int(valor.split(' - ')[0])
    return valor

def jogos_fora_gols_marcados(valor):
    if '-' in valor:
        return int(valor.split(' - ')[1])
    return valor

def jogos_em_casa_gols_sofridos(valor):
    if '-' in valor:
        return int(valor.split(' - ')[1])
    return valor

def jogos_fora_gols_sofrido(valor):
    if '-' in valor:
        return int(valor.split(' - ')[0])
    return valor

def resultados(time):    
    driver = webdriver.Chrome()
    #endereco = endereco_estatistica + time
    endereco = time

    driver.get(endereco)
    ttime.sleep(1)    
    clicar(driver,'/html/body/div[1]/div/div/div/div[2]/div/button[3]') # agree

    xpath_b = '/html/body/div[3]/div[2]/div[5]/div/div[2]/table[1]/tbody/tr/td/table/tbody'
    linha_b = 'tr'
    coluna_b = 'td'

    ttime.sleep(1)
    
    df = pd.DataFrame(columns = ['Pos','Time','GP','Pts','Home','Away','h2h'])

    #Body
    #tabela = driver.find_element("xpath",xpath_b)
    tabela = driver.find_element('xpath',"//h2[text()='Results table']/following-sibling::table[1]")
    
    
    linhas = tabela.find_elements(By.TAG_NAME,linha_b)

    for linha in linhas:
        colunas = linha.find_elements(By.TAG_NAME,coluna_b)
        col = []
        for coluna in colunas:
            col.append(coluna.text)  
        col.append('')
        col.append('')
        col.append('')
        df_insert = pd.DataFrame(data={'Pos':[col[0]],
                                          'Time':[col[1]],
                                          'GP':[col[2]],
                                          'Pts':[col[3]],
                                          'Home':[col[4]],
                                          'Away':[col[5]],
                                          'h2h':[col[6]]})     
        df = pd.concat([df, df_insert], ignore_index=True)

    df = df.drop(df.index[0])

    df['Home'] = df['Home'].apply(lambda x: x if '-' in str(x) else '')
    df['Away'] = df['Away'].apply(lambda x: x if '-' in str(x) else '')

    df = df.set_index('Pos')

    df = df[['Time','GP','Pts','Home','Away']]

    df['casa_Marcados'] = df['Home']
    df['casa_Marcados'] = df['casa_Marcados'].apply(lambda x: jogos_em_casa_gols_marcados(x))
    df['casa_Sofridos'] = df['Home']
    df['casa_Sofridos'] = df['casa_Sofridos'].apply(lambda x: jogos_em_casa_gols_sofridos(x))

    df['vis_Marcados'] = df['Away']
    df['vis_Marcados'] = df['vis_Marcados'].apply(lambda x: jogos_fora_gols_marcados(x))
    df['vis_Sofridos'] = df['Away']
    df['vis_Sofridos'] = df['vis_Sofridos'].apply(lambda x: jogos_fora_gols_sofrido(x))

    df = df.set_index('Time')
    df['casa_Marcados'] = df['casa_Marcados'].apply(pd.to_numeric, errors='coerce')
    df['casa_Sofridos'] = df['casa_Sofridos'].apply(pd.to_numeric, errors='coerce')
    df['vis_Marcados'] = df['vis_Marcados'].apply(pd.to_numeric, errors='coerce')
    df['vis_Sofridos'] = df['vis_Sofridos'].apply(pd.to_numeric, errors='coerce')

    dic = {'casa_marcado_sup':round(df['casa_Marcados'].head(10).mean(skipna=True),2),
           'casa_marcado_sup_std':round(df['casa_Marcados'].head(10).std(skipna=True),4),
           'casa_marcado_inf':round(df['casa_Marcados'].tail(10).mean(skipna=True),2),
           'casa_marcado_inf_std':round(df['casa_Marcados'].tail(10).std(skipna=True),4),
           'casa_sofrido_sup':round(df['casa_Sofridos'].head(10).mean(skipna=True),2),
           'casa_sofrido_sup_std':round(df['casa_Sofridos'].head(10).std(skipna=True),4),
           'casa_sofrido_inf':round(df['casa_Sofridos'].tail(10).mean(skipna=True),2),
           'casa_sofrido_inf_std':round(df['casa_Sofridos'].tail(10).std(skipna=True),4),
           'vis_marcado_sup':round(df['vis_Marcados'].head(10).mean(skipna=True),2),
           'vis_marcado_sup_std':round(df['vis_Marcados'].head(10).std(skipna=True),4),
           'vis_marcado_inf':round(df['vis_Marcados'].tail(10).mean(skipna=True),2),
           'vis_marcado_inf_std':round(df['vis_Marcados'].tail(10).std(skipna=True),4),
           'vis_sofrido_sup':round(df['vis_Sofridos'].head(10).mean(skipna=True),2),
           'vis_sofrido_sup_std':round(df['vis_Sofridos'].head(10).std(skipna=True),4),
           'vis_sofrido_inf':round(df['vis_Sofridos'].tail(10).mean(skipna=True),2),
           'vis_sofrido_inf_std':round(df['vis_Sofridos'].tail(10).std(skipna=True),4)}

    return dic
    
def ler_tabela__sem_c(endereco,mes,linha_b,coluna_b):
        driver = webdriver.Chrome()
        driver.get(endereco)
        ttime.sleep(1)
        
        clicar(driver,'/html/body/div[1]/div/div/div/div[2]/div/button[3]') # agree


        #Body
        #tabela = driver.find_element("xpath",xpath_b)
        tabela = driver.find_element('xpath',f"//h2[text()='{mes}']/following-sibling::table[1]")
        linhas = tabela.find_elements(By.TAG_NAME,linha_b)

        df_matches = pd.DataFrame(columns = ['Data', 'Time_C', 'Hora', 'Time_V'] )

        for linha in linhas:
            colunas = linha.find_elements(By.TAG_NAME,coluna_b)
            col = []
            for coluna in colunas:
                col.append(coluna.text)      

            if len(col)>3:
                try:
                    col = [elemento for elemento in col if elemento != '']
                    df_insert = pd.DataFrame(data={'Data':[col[0]],
                                                   'Time_C':[col[1]],
                                                   'Hora':[col[2]],
                                                   'Time_V':[col[3]]})

                    df_matches = pd.concat([df_matches, df_insert], ignore_index=True)
                except Exception as e:
                    #print(e)
                    pass

                    

        df_matches = df_matches.loc[df_matches['Hora'].str.contains(':')==True]

        #df_matches = df_matches.drop(df_matches.index[0])        

        df_matches['Time_C'] = df_matches['Time_C'].apply(lambda x: x.lstrip().rstrip()) 
        df_matches['Time_V'] = df_matches['Time_V'].apply(lambda x: x.lstrip().rstrip()) 

        return df_matches
    
def probabilidade(c,c_std,v,v_std,n):
    #casa = np.random.exponential(scale=c, size=n)
    #vis = np.random.exponential(scale=v, size=n)
    casa = []
    vis = []
    for i in range(0,n):
        casa.append(random.normalvariate(c, c_std))
        vis.append(random.normalvariate(v, v_std))     


    m = 100/n
    c = 0.0
    e = 0.0
    v = 0.0

    for i in range(0,n):
        if (casa[i] - vis[i]) > 0.5:
            c += m
        elif (casa[i] - vis[i]) < -0.5:
            v += m
        else:
            e += m

    return int(c),int(e),int(v)

In [4]:
def faz_leitura(endereco_o,endereco_estatistica,endereco_resultado,mes):    
    

    driver = webdriver.Chrome()
    driver.get(endereco_o)
    
    ttime.sleep(1)
    clicar(driver,'/html/body/div[1]/div/div/div/div[2]/div/button[3]') # agree


    xpath_b = '/html/body/div[3]/div[2]/div[4]/div[4]/table[3]'
    linha_b = 'tr'
    coluna_b = 'td'

    ttime.sleep(1)
    #Body
    tabela = driver.find_element('xpath',"//h2[text()='Table']/following-sibling::table[1]")

    linhas = tabela.find_elements(By.TAG_NAME,linha_b)

    times = {}
    df_clas = pd.DataFrame(columns = ['Pos','Time','GP','W','D','L','GF','GA','GD','Pts','PPG','last8','CS','FTS'])
    for linha in linhas:
        colunas = linha.find_elements(By.TAG_NAME,coluna_b)
        links = linha.find_elements(By.TAG_NAME,'a')
        for link in links:
            aux = link.get_attribute('href')
            txt = link.text
            if 'stats=' in aux:
                times[txt]=aux

        col = []
        for coluna in colunas:
            col.append(coluna.text)  
        if len(col)>10:
            col = [elemento for elemento in col if elemento != '']

            df_insert = pd.DataFrame(data={'Pos':[col[0]],
                                           'Time':[col[1]],
                                           'GP':[col[2]],
                                           'W':[col[3]],
                                           'D':[col[4]],
                                           'L':[col[5]],
                                           'GF':[col[6]],
                                           'GA':[col[7]],
                                           'GD':[col[8]],
                                           'Pts':[col[9]],
                                           'PPG':[col[10]],
                                           'last8':[col[11]],
                                           'CS':[col[12]],
                                           'FTS':[col[13]]})
            df_clas = pd.concat([df_clas, df_insert], ignore_index=True)

    df_clas = df_clas.drop(df_clas.index[0])        

    df_clas['Time'] = df_clas['Time'].apply(lambda x: x.lstrip().rstrip()) 

    #display(df_clas)
    
    df_resultados = pd.DataFrame(columns = ['casa_marcado_sup','casa_marcado_inf','casa_sofrido_sup','casa_sofrido_inf','vis_marcado_sup','vis_marcado_inf','vis_sofrido_sup','vis_sofrido_inf'])

    todos = df_clas['Time'].to_list()
    #todos = todos[:2]


    i = 0
    cont = 0
    while i < len(todos):
        t = todos[i]
        try:
            if cont < 5:
                df_insert = pd.DataFrame(resultados(times[t]),index=[t])
                df_resultados = pd.concat([df_resultados, df_insert])
                i += 1
                cont = 0
            else:
                break
        except Exception as e:
            print('Não funfou',t,cont,times[t])
            print(e)
            cont += 1


    
    print(endereco_resultado)
    
    endereco = endereco_resultado
    xpath_b = '/html/body/div[3]/div[2]/div[7]/table[1]'
    linha_b = 'tr'
    coluna_b = 'td'
    df_matches = ler_tabela__sem_c(endereco,mes,linha_b,coluna_b)

    #display(df_matches)
    
    df = df_matches.copy()
    df['Casa_marcado'] = 0
    df['Casa_marcado_std'] = 0   
    df['Casa_sofrido'] = 0
    df['Casa_sofrido_std'] = 0
    
    df['Visitante_marcado'] = 0
    df['Visitante_marcado_std'] = 0
    df['Visitante_sofrido'] = 0
    df['Visitante_sofrido_std'] = 0
    df['C'] = 0
    df['V'] = 0

    #display(df_clas)
    
    
    for i,row in df.iterrows():
        try:
            df.loc[df['Time_C']==row['Time_C'],'C'] = int(df_clas.loc[df_clas['Time']==row['Time_C'],'Pos'].to_list()[0])
            df.loc[df['Time_V']==row['Time_V'],'V'] = int(df_clas.loc[df_clas['Time']==row['Time_V'],'Pos'].to_list()[0])
        except:
            print('Falha no',row['Time_C'],' x ',row['Time_V'])

    #poe o valor medio e o desvio padrão do time
    for i,row in df.iterrows():
        try:
            df.loc[(df['Time_C']==row['Time_C'])&(df['V']<10),'Casa_marcado'] = df_resultados.loc[row['Time_C'],'casa_marcado_sup']
            df.loc[(df['Time_C']==row['Time_C'])&(df['V']>=10),'Casa_marcado'] = df_resultados.loc[row['Time_C'],'casa_marcado_inf']  
            df.loc[(df['Time_C']==row['Time_C'])&(df['V']<10),'Casa_sofrido'] = df_resultados.loc[row['Time_C'],'casa_sofrido_sup']
            df.loc[(df['Time_C']==row['Time_C'])&(df['V']>=10),'Casa_sofrido'] = df_resultados.loc[row['Time_C'],'casa_sofrido_inf']

            df.loc[(df['Time_V']==row['Time_V'])&(df['C']<10),'Visitante_marcado'] = df_resultados.loc[row['Time_V'],'vis_marcado_sup']
            df.loc[(df['Time_V']==row['Time_V'])&(df['C']>=10),'Visitante_marcado'] = df_resultados.loc[row['Time_V'],'vis_marcado_inf']  
            df.loc[(df['Time_V']==row['Time_V'])&(df['C']<10),'Visitante_sofrido'] = df_resultados.loc[row['Time_V'],'vis_sofrido_sup']
            df.loc[(df['Time_V']==row['Time_V'])&(df['C']>=10),'Visitante_sofrido'] = df_resultados.loc[row['Time_V'],'vis_sofrido_inf']


            df.loc[(df['Time_C']==row['Time_C'])&(df['V']<10),'Casa_marcado_std'] = df_resultados.loc[row['Time_C'],'casa_marcado_sup_std']
            df.loc[(df['Time_C']==row['Time_C'])&(df['V']>=10),'Casa_marcado_std'] = df_resultados.loc[row['Time_C'],'casa_marcado_inf_std']  
            df.loc[(df['Time_C']==row['Time_C'])&(df['V']<10),'Casa_sofrido_std'] = df_resultados.loc[row['Time_C'],'casa_sofrido_sup_std']
            df.loc[(df['Time_C']==row['Time_C'])&(df['V']>=10),'Casa_sofrido_std'] = df_resultados.loc[row['Time_C'],'casa_sofrido_inf_std']

            df.loc[(df['Time_V']==row['Time_V'])&(df['C']<10),'Visitante_marcado_std'] = df_resultados.loc[row['Time_V'],'vis_marcado_sup_std']
            df.loc[(df['Time_V']==row['Time_V'])&(df['C']>=10),'Visitante_marcado_std'] = df_resultados.loc[row['Time_V'],'vis_marcado_inf_std']  
            df.loc[(df['Time_V']==row['Time_V'])&(df['C']<10),'Visitante_sofrido_std'] = df_resultados.loc[row['Time_V'],'vis_sofrido_sup_std']
            df.loc[(df['Time_V']==row['Time_V'])&(df['C']>=10),'Visitante_sofrido_std'] = df_resultados.loc[row['Time_V'],'vis_sofrido_inf_std']
        except:
            print('Falha no',row['Time_C'],' x ',row['Time_V'])
    
    #display(df)
    
    df = df[['Data','Hora','C','Time_C','V','Time_V','Casa_marcado','Visitante_sofrido','Visitante_marcado','Casa_sofrido','Casa_marcado_std','Visitante_sofrido_std','Visitante_marcado_std','Casa_sofrido_std']]
    df = df.set_index(['Data','Hora'])
    df_matches = df.copy()

    df_prob = df_matches.copy()
    df_prob['p_c_atq'] = 0
    df_prob['p_c_def'] = 0
    df_prob['p_e_atq'] = 0
    df_prob['p_e_def'] = 0
    df_prob['p_v_atq'] = 0
    df_prob['p_v_def'] = 0

    #calcula a probabilidade 
    for i,row in df_prob.iterrows():
        c,e,v = probabilidade(row['Casa_marcado'],row['Casa_marcado_std'],row['Visitante_marcado'],row['Visitante_marcado_std'],10000)  
        df_prob.loc[(df_prob['Time_C']==row['Time_C'])&(df_prob['Time_V']==row['Time_V']),'p_c_atq'] = c
        df_prob.loc[(df_prob['Time_C']==row['Time_C'])&(df_prob['Time_V']==row['Time_V']),'p_e_atq'] = e
        df_prob.loc[(df_prob['Time_C']==row['Time_C'])&(df_prob['Time_V']==row['Time_V']),'p_v_atq'] = v
        c,e,v = probabilidade(row['Visitante_sofrido'],row['Visitante_sofrido_std'],row['Casa_sofrido'],row['Casa_sofrido_std'],10000)  
        df_prob.loc[(df_prob['Time_C']==row['Time_C'])&(df_prob['Time_V']==row['Time_V']),'p_c_def'] = c
        df_prob.loc[(df_prob['Time_C']==row['Time_C'])&(df_prob['Time_V']==row['Time_V']),'p_e_def'] = e
        df_prob.loc[(df_prob['Time_C']==row['Time_C'])&(df_prob['Time_V']==row['Time_V']),'p_v_def'] = v


    df_prob['Casa'] = (df_prob['p_c_atq'] + df_prob['p_c_def'])/2
    df_prob['Empate'] = (df_prob['p_e_atq'] + df_prob['p_e_def'])/2
    df_prob['Visitante'] = (df_prob['p_v_atq'] + df_prob['p_v_def'])/2

    df_prob = df_prob[['C','Time_C','V','Time_V','Casa','Empate','Visitante',
                            'p_c_atq','p_c_def','p_e_atq','p_e_def','p_v_atq','p_v_def']]


    df = df_prob.copy()
    df = df.loc[df['Casa']>60]
    
    df = df.reset_index()
    df['Data'] = df['Data'].apply(lambda x: int(x.split(' ')[1]))
    df = df.loc[df['Data']<df['Data'].min()+5] # 5 dias a mais a partir da data menor que aparece nos proximos jogos
    df = df.set_index('Data')

    display(df)
    return df

In [ ]:
campeonatos = ['brazil','brazil2','brazil3','argentina','argentina3','chile','japan','norway','sweden']
#campeonatos = ['brazil3']
#campeonatos = ['norway']
#campeonatos = ['chile']
campeonatos = ['china']



for campeonato in campeonatos:
    print(campeonato)
    mes = 'August'
    endereco_o = 'https://www.soccerstats.com/latest.asp?league=' + campeonato
    endereco_estatistica = 'https://www.soccerstats.com/team.asp?league=' + campeonato +'&stats='
    endereco_resultado = 'https://www.soccerstats.com/results.asp?league=' + campeonato + '&pmtype=month8'
    
    try:
        faz_leitura(endereco_o,endereco_estatistica,endereco_resultado,mes)
    except Exception as e:
        print('Não funfou o ',campeonato)
        print(e)
    
    
    

brazil
https://www.soccerstats.com/results.asp?league=brazil&pmtype=month8


,Hora,C,Time_C,V,Time_V,Casa,Empate,Visitante,p_c_atq,p_c_def,p_e_atq,p_e_def,p_v_atq,p_v_def
Data,,,,,,,,,,,,,,
5,21:30,12,Internacional,14,Corinthians,58.5,24.5,15.0,64,53,26,23,8,22
6,19:00,8,Sao Paulo,13,Atletico MG,50.5,23.5,24.5,63,38,14,33,21,28
6,21:30,17,Bahia,19,America MG,62.5,13.0,23.0,29,96,23,3,46,0


brazil2
https://www.soccerstats.com/results.asp?league=brazil2&pmtype=month8


,Hora,C,Time_C,V,Time_V,Casa,Empate,Visitante,p_c_atq,p_c_def,p_e_atq,p_e_def,p_v_atq,p_v_def
Data,,,,,,,,,,,,,,
1,22:00,7,Mirassol,18,Avai,62.5,20.5,15.5,56,69,24,17,18,13
2,22:00,6,Criciuma,13,Ponte Preta,87.5,7.5,4.0,98,77,1,14,0,8
3,00:30,5,Guarani,10,Ceara,54.5,25.0,19.0,63,46,19,31,16,22
3,00:30,19,Londrina,17,Chapecoense,63.0,14.5,21.5,40,86,19,10,40,3
3,00:30,4,Vila Nova,2,Sport Recife,51.5,36.0,11.0,63,40,30,42,6,16
3,00:30,3,Vitoria,20,ABC,66.5,21.0,10.5,60,73,29,13,9,12
5,00:30,7,Mirassol,15,Sampaio Correa,71.5,19.5,7.0,72,71,19,20,7,7


brazil3
https://www.soccerstats.com/results.asp?league=brazil3&pmtype=month8


...

chile
https://www.soccerstats.com/results.asp?league=chile&pmtype=month8
Falha no U. de Chile  x  O'Higgins
Falha no O'Higgins  x  Copiapo
Falha no Curico Unido  x  O'Higgins
Falha no U. de Chile  x  O'Higgins
Falha no O'Higgins  x  Copiapo
Falha no Curico Unido  x  O'Higgins


,Hora,C,Time_C,V,Time_V,Casa,Empate,Visitante,p_c_atq,p_c_def,p_e_atq,p_e_def,p_v_atq,p_v_def
Data,,,,,,,,,,,,,,
5,21:30,9,U. Catolica,12,Audax Italiano,64.5,19.0,15.5,70,59,14,24,15,16


japan
Não funfou S. Hiroshima 0 https://www.soccerstats.com/team.asp?league=japan&stats=13-s.-hiroshima
'NoneType' object has no attribute 'find_elements'
https://www.soccerstats.com/results.asp?league=japan&pmtype=month8


,Hora,C,Time_C,V,Time_V,Casa,Empate,Visitante,p_c_atq,p_c_def,p_e_atq,p_e_def,p_v_atq,p_v_def
Data,,,,,,,,,,,,,,
6,09:00,6,Kashima Antlers,11,C. Sapporo,56.5,18.0,23.5,39,74,20,16,39,8
6,10:00,7,K. Frontale,13,Gamba Osaka,78.0,14.0,6.5,69,87,20,8,10,3
6,10:00,15,Kyoto Sanga,17,Kashiwa Reysol,60.0,19.0,20.0,84,36,14,24,1,39


norway


In [6]:
campeonatos = ['china']
campeonatos = ['southkorea']

for campeonato in campeonatos:
    print(campeonato)
    mes = 'August'
    endereco_o = 'https://www.soccerstats.com/latest.asp?league=' + campeonato
    endereco_estatistica = 'https://www.soccerstats.com/team.asp?league=' + campeonato +'&stats='
    endereco_resultado = 'https://www.soccerstats.com/results.asp?league=' + campeonato + '&pmtype=month8'
    
    try:
        faz_leitura(endereco_o,endereco_estatistica,endereco_resultado,mes)
    except Exception as e:
        print('Não funfou o ',campeonato)
        print(e)

southkorea
Não funfou Ulsan 0 https://www.soccerstats.com/team.asp?league=southkorea&stats=1-ulsan
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//h2[text()='Results table']/following-sibling::table[1]"}
  (Session info: chrome=115.0.5790.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00C7A813+48355]
	(No symbol) [0x00C0C4B1]
	(No symbol) [0x00B15358]
	(No symbol) [0x00B409A5]
	(No symbol) [0x00B40B3B]
	(No symbol) [0x00B6E232]
	(No symbol) [0x00B5A784]
	(No symbol) [0x00B6C922]
	(No symbol) [0x00B5A536]
	(No symbol) [0x00B382DC]
	(No symbol) [0x00B393DD]
	GetHandleVerifier [0x00EDAABD+2539405]
	GetHandleVerifier [0x00F1A78F+2800735]
	GetHandleVerifier [0x00F1456C+2775612]
	GetHandleVerifier [0x00D051E0+616112]
	(No symbol) [0x00C15F8C]
	(No symbol) [0x00C12328]
	(No symbol) [0x00C1240B]
	(No symbol) [0x00C04FF7]
	BaseThreadInitThunk [0x75C57D59+25]
	RtlInitializeExceptionChain [0x770BB79B+107]
	RtlClearBits [0x770BB71F+191]

Não funfou Ulsan 1 ht

,Hora,C,Time_C,V,Time_V,Casa,Empate,Visitante,p_c_atq,p_c_def,p_e_atq,p_e_def,p_v_atq,p_v_def
Data,,,,,,,,,,,,,,
